In [3]:
# Much of the work done here based on 
# https://www.kaggle.com/jazivxt/courtside-seat-2019m-competitiveness
# Using as template starter kernel to get off the ground with first attempt
import sklearn
import os
import pandas as pd # data processing
import numpy as np # linear algebra analysis
import matplotlib.pyplot as plt
import seaborn as sns # visualization
from sklearn import *
import warnings
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")

In [4]:
PATH = '/Users/kelson/Kaggle/MarchMadness/Data/'

In [5]:
# Read in important files 
teams = pd.read_csv(f'{PATH}datafiles/Teams.csv')

# Specify encoding. File throws Unicode parsing error otherwise
team_spellings = pd.read_csv(f'{PATH}datafiles/TeamSpellings.csv', encoding='latin-1')

season_compact = pd.read_csv(f'{PATH}datafiles/RegularSeasonCompactResults.csv')
season_detailed = pd.read_csv(f'{PATH}datafiles/RegularSeasonDetailedResults.csv')
tourney_compact = pd.read_csv(f'{PATH}datafiles/NCAATourneyCompactResults.csv')
tourney_detailed = pd.read_csv(f'{PATH}datafiles/NCAATourneyDetailedResults.csv')

seeds = pd.read_csv(f'{PATH}datafiles/NCAATourneySeeds.csv')
# Reformat seeds to be numerical 
seeds['seed_num'] = seeds.Seed.apply(lambda x: int(x[1:3]))


In [6]:
# Read in Massey Ordinals
mo = pd.read_csv(f'{PATH}MasseyOrdinals.csv')
mo.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [7]:
# Read in sample submission to modify
sub = pd.read_csv(f'{PATH}SampleSubmissionStage1.csv')

In [8]:
# Count how many different times a certain TeamID is associated with a different spelling
team_spellings = team_spellings.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
# Rename column intuitively 
team_spellings.columns = ['TeamID', 'TeamNameCount']
team_spellings.head()

,TeamID,TeamNameCount
0,1101,3
1,1102,2
2,1103,1
3,1104,1
4,1105,2


In [9]:
team_spellings.head()

,TeamID,TeamNameCount
0,1101,3
1,1102,2
2,1103,1
3,1104,1
4,1105,2


In [10]:
# Merge dfs so that main teams df has TeamNameCount
teams = pd.merge(teams, team_spellings, how='left', on=['TeamID'])
# No longer need team_spellings
del team_spellings

In [11]:
# Add a tag to distinguish seasonal vs tournament games
season_compact['ST'] = 'S'
season_detailed['ST'] = 'S'
tourney_compact['ST'] = 'T'
tourney_detailed['ST'] = 'T'

# Merge all games 
# For now, keep model simple and only use compact Tourney results
# games = pd.concat((season_compact, tourney_compact), axis=0, ignore_index=True)
games = tourney_compact
# games.reset_index(drop=True, inplace=True)

# Map win location to a numerical value
# Uncomment if using regular season games as well
# games['WLoc'] = games['WLoc'].map({'A':1, 'H':2, 'N':3})

In [12]:
# Conjoin season, Team1, and Team2 for competition standard ID format
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ID
0,1985,136,1116,63,1234,54,N,0,T,1985_1116_1234
1,1985,136,1120,59,1345,58,N,0,T,1985_1120_1345
2,1985,136,1207,68,1250,43,N,0,T,1985_1207_1250
3,1985,136,1229,58,1425,55,N,0,T,1985_1229_1425
4,1985,136,1242,49,1325,38,N,0,T,1985_1242_1325


In [13]:
# Rename TeamID columns for competition format
# When the team with the lower ID# won, set WTeam to Team 1
games['Team1'] = np.where((games.WTeamID < games.LTeamID), games.WTeamID, games.LTeamID)
games['Team2'] = np.where((games.WTeamID > games.LTeamID), games.WTeamID, games.LTeamID)

# Add target feature - binary value of win or loss
# Target represents Team1 winning
games['target'] = np.where((games['Team1'] < games['Team2']), 1, 0)
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ID,Team1,Team2,target
0,1985,136,1116,63,1234,54,N,0,T,1985_1116_1234,1116,1234,1
1,1985,136,1120,59,1345,58,N,0,T,1985_1120_1345,1120,1345,1
2,1985,136,1207,68,1250,43,N,0,T,1985_1207_1250,1207,1250,1
3,1985,136,1229,58,1425,55,N,0,T,1985_1229_1425,1229,1425,1
4,1985,136,1242,49,1325,38,N,0,T,1985_1242_1325,1242,1325,1


In [14]:
# Add seed values to dataframe 
games = games.merge(seeds, how='left', left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
games = games.merge(seeds, how='left', left_on=['Season', 'Team2'], right_on=['Season', 'TeamID'], suffixes=('Team1', 'Team2'))
games.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ID,Team1,Team2,target,SeedTeam1,TeamIDTeam1,seed_numTeam1,SeedTeam2,TeamIDTeam2,seed_numTeam2
0,1985,136,1116,63,1234,54,N,0,T,1985_1116_1234,1116,1234,1,X09,1116,9,X08,1234,8
1,1985,136,1120,59,1345,58,N,0,T,1985_1120_1345,1120,1345,1,Z11,1120,11,Z06,1345,6
2,1985,136,1207,68,1250,43,N,0,T,1985_1207_1250,1207,1250,1,W01,1207,1,W16,1250,16
3,1985,136,1229,58,1425,55,N,0,T,1985_1229_1425,1229,1425,1,Y09,1229,9,Y08,1425,8
4,1985,136,1242,49,1325,38,N,0,T,1985_1242_1325,1242,1325,1,Z03,1242,3,Z14,1325,14


In [15]:
# Add Massey Ordinal data to dataframe
# Team1
games = games.merge(mo, left_on=['Season', 'Team1'], right_on=['Season', 'TeamID'])
# Team2
games = games.merge(mo, left_on=['Season', 'Team2', 'SystemName'], right_on=['Season', 'TeamID', 'SystemName'], suffixes=['Team1', 'Team2'])
games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ST,ID,...,SeedTeam2,TeamIDTeam2,seed_numTeam2,RankingDayNumTeam1,SystemName,TeamIDTeam1,OrdinalRankTeam1,RankingDayNumTeam2,TeamIDTeam2,OrdinalRankTeam2
0,2003,134,1421,92,1411,84,N,1,T,2003_1411_1421,...,X16b,1421,16,35,SEL,1411,231,35,1421,218
1,2003,134,1421,92,1411,84,N,1,T,2003_1411_1421,...,X16b,1421,16,35,SEL,1411,231,43,1421,223
2,2003,134,1421,92,1411,84,N,1,T,2003_1411_1421,...,X16b,1421,16,35,SEL,1411,231,49,1421,211
3,2003,134,1421,92,1411,84,N,1,T,2003_1411_1421,...,X16b,1421,16,35,SEL,1411,231,56,1421,223
4,2003,134,1421,92,1411,84,N,1,T,2003_1411_1421,...,X16b,1421,16,35,SEL,1411,231,65,1421,208


In [20]:
# Combine score, seed, and Massey Ordinal difference between teams
games['ScoreDiff'] = games['WScore'] - games['LScore']
# Normalize to account for when Team1 isn't the winning team
games['SeedDiff'] = games['seed_numTeam1'] - games['seed_numTeam2']
games['MoDiff'] = games['OrdinalRankTeam1'] - games['OrdinalRankTeam2']


In [21]:
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['target'] == 0. else r['ScoreDiff'], axis=1)
